# Database and SQL course
Welcome to introduction to Database and SQL

Structure Query Language(SQL, usually pronounced S-Q-L or 'sequel') is a programming language designed for the data manipulation with database system, mostly with relational DBMS, like MySQL, PostgreSQL and Oracle

In this course, we will act as a data analyst for a fictional superstore. Our job is to transform raw data into information and clean it to the tabluar format, and do the data exploration with different business purpose, prepare the story-telling for the stakeholders.

In this folder, we have the two csv files, we will use python to create a basic sales database and use the SQL to query the data. And gradually we will advance to more complex queries with advance data anlysis skills like data wrangling, data filtering, sorting, grouping and statstical analysis.

By the end of this course, you will be able to write queries like the one shown, which can return a list of the sales data, can compare different store, different category goods, their sales volume and profits.

Don't worry if it seems daunting now; we'll master it step by step;

Notes: this course focus on the SQL, so some background knowledge on the jupyter notebook and pandas etc, please see the resource on the web.


# Prepare the data

First we need import the necessary python package

In [14]:
import numpy as np
import pandas as pd
import duckdb
sql = lambda s: duckdb.sql(s)

## Load the sales data

In [16]:
df1 = pd.read_csv('./orders.csv')
df2 = pd.read_csv('./details.csv')
sales_df = pd.merge(df1,df2,on='Order ID')
sales_df.head(3)

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode
0,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,5729,64,14,Furniture,Chairs,EMI
1,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,671,114,9,Electronics,Phones,Credit Card
2,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,443,11,1,Clothing,Saree,COD


## Do some data type tranformation

### Transform the column into the table compatible format

In [21]:
sales_df.columns = df.columns.map(lambda c: '_'.join(c.lower().split(' ')))
sales_df.columns

Index(['order_id', 'order_date', 'customername', 'state', 'city', 'amount',
       'profit', 'quantity', 'category', 'sub-category', 'paymentmode'],
      dtype='object')

### Convert the order date the pandas datetime data type

In [23]:
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'],format='%d-%m-%Y')

In [24]:
sales_df.head(3)

,order_id,order_date,customername,state,city,amount,profit,quantity,category,sub-category,paymentmode
0,B-26055,2018-03-10,Harivansh,Uttar Pradesh,Mathura,5729,64,14,Furniture,Chairs,EMI
1,B-26055,2018-03-10,Harivansh,Uttar Pradesh,Mathura,671,114,9,Electronics,Phones,Credit Card
2,B-26055,2018-03-10,Harivansh,Uttar Pradesh,Mathura,443,11,1,Clothing,Saree,COD


## Query the data with the duckdb sql capability

In [26]:
sql('select * from sales_df limit 3')

┌──────────┬─────────────────────┬──────────────┬───┬──────────┬─────────────┬──────────────┬─────────────┐
│ order_id │     order_date      │ customername │ … │ quantity │  category   │ sub-category │ paymentmode │
│ varchar  │    timestamp_ns     │   varchar    │   │  int64   │   varchar   │   varchar    │   varchar   │
├──────────┼─────────────────────┼──────────────┼───┼──────────┼─────────────┼──────────────┼─────────────┤
│ B-26055  │ 2018-03-10 00:00:00 │ Harivansh    │ … │       14 │ Furniture   │ Chairs       │ EMI         │
│ B-26055  │ 2018-03-10 00:00:00 │ Harivansh    │ … │        9 │ Electronics │ Phones       │ Credit Card │
│ B-26055  │ 2018-03-10 00:00:00 │ Harivansh    │ … │        1 │ Clothing    │ Saree        │ COD         │
├──────────┴─────────────────────┴──────────────┴───┴──────────┴─────────────┴──────────────┴─────────────┤
│ 3 rows                                                                             11 columns (7 shown) │
└───────────────────────────

## First SQL we try to say hello to database
Now the duckdb database has only one table sales_df, which contains information about the orders from the superstore stores, our first line of code is to retrieve the first 3 rows of this tables:
```sql
SELECT *
    FROM sales_df
    LIMIT 3;
```
Let's explain this code a little further:
 - \* is a special character meaning 'all the column of the table'. We will learn more about the SELECtclause later, but for now we can read this line as 'Select all columns'
 - FROM clause tell database which table that we want to retrieve the data. We will learn more about how to identify the database tables and how to manage them.
 - LIMIT 3 tell the database the result return from limit to 3 records. A table might have millions rows of data, attempting to return all the data could get messy.
 

### Different SQL Dialects
There are many different versions of SQL(also called SQL flavors or diablects(. We have benn coding against DuckDB, which is complied to the ANSI SQL standard, other database like MySQL and Oracle Database also support the ANSI SQL standard, but they all have their own dialects to empower their capability, we need take an eyes on the difference when we work on the specific database engine.

![sql engine](./images/sql_engine.png)


### Database tables and fields
Most database we will work with are like superstore database has multiple tables of data.
Like a sheet in Excel, a database table consists of rows and columns which are offen called records and fields.

![database tables](./images/tables.png)

### First Example: Select all columns from sales table
```sql
SELECT * 
    FROM sales_df 
    LIMIT 3;
```
As we introduce the say hello to the database tables, we can select all the columns from the database tables;

But what about reducing the number of fields shown? As we learned earlier, we can retrieve the data from all data field in a table with help of the \* wildcard. If we want to select the specific fields to be return by the query, we can list them in the SELECT clause, seperated by commas:

```sql
SELECT order_date, customername, category 
    FROM sales_df
    LIMIT 5;
```

In [27]:
sql('select order_date, customername, category from sales_df limit 5')

┌─────────────────────┬──────────────┬─────────────┐
│     order_date      │ customername │  category   │
│    timestamp_ns     │   varchar    │   varchar   │
├─────────────────────┼──────────────┼─────────────┤
│ 2018-03-10 00:00:00 │ Harivansh    │ Furniture   │
│ 2018-03-10 00:00:00 │ Harivansh    │ Electronics │
│ 2018-03-10 00:00:00 │ Harivansh    │ Clothing    │
│ 2018-03-10 00:00:00 │ Harivansh    │ Clothing    │
│ 2018-03-10 00:00:00 │ Harivansh    │ Clothing    │
└─────────────────────┴──────────────┴─────────────┘